In [ ]:
from collections import defaultdict
from itertools import chain, combinations, product

import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
def get_cycle(model, start_node):

    i = start_node
    sol_edges = {e for e in model.E if model.x[e].value > 0.5}
    path_nodes = [i]
    path_edges = list()

    while True:
        for (a, b) in sol_edges:
            if i in (a, b) and model.x[a, b].value == 1:
                if a == i:
                    path_nodes.append(b)
                    path_edges.append((a, b))
                    sol_edges.remove((a, b))
                    i = b
                    break
                if b == i:
                    path_nodes.append(a)
                    path_edges.append((a, b))
                    sol_edges.remove((a, b))
                    i = a
                    break

        if len(path_nodes) > 1 and path_nodes[-1] == start_node:
            break
            
    return path_edges

In [ ]:
V = {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}
E = {('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'G'),
     ('B', 'E'), ('B', 'F'), ('C', 'G'), ('D', 'E'),
     ('D', 'G'), ('E', 'H'), ('F', 'H'), ('G', 'H')}
c = {('A', 'B'): 5, ('A', 'C'): 1, ('A', 'D'): 2, ('A', 'G'): 1,
     ('B', 'E'): 2, ('B', 'F'): 1, ('C', 'G'): 1, ('D', 'E'): 1,
     ('D', 'G'): 2, ('E', 'H'): 2, ('F', 'H'): 1, ('G', 'H'): 4}

In [ ]:
E_of_i = defaultdict(set)
for (i, j) in E:
    E_of_i[i].add((i, j))
    E_of_i[j].add((i, j))

# Approach \#1
Use a `ConstraintList`. Iterate over sets in the power set of nodes.

In [ ]:
model = pe.ConcreteModel()

model.V = pe.Set(initialize=V)
model.E = pe.Set(initialize=E)
model.c = pe.Param(model.E, initialize=c)
model.E_of_i = pe.Param(model.V, initialize=E_of_i)

model.x = pe.Var(model.E, domain=pe.Reals, bounds=(0, 1))

obj_expr = sum(model.c[e] * model.x[e] for e in model.E)
model.obj = pe.Objective(sense=pe.minimize, expr=obj_expr)

def con_flow_balance(model, i):
    return sum(model.x[e] for e in model.E_of_i[i]) == 2
model.con_flow_balance = pe.Constraint(model.V, rule=con_flow_balance)

model.con_no_subtours = pe.ConstraintList()
for S in powerset(model.V):
    if 2 <= len(S) <= len(model.V) - 1:
        E_of_S = set((a, b)
                     for i in S
                     for (a, b) in model.E_of_i[i]
                     if a in S and b in S)
        if E_of_S:
            lhs_expr = sum(model.x[e] for e in E_of_S)
            rhs_expr = len(S) - 1
            model.con_no_subtours.add(lhs_expr <= rhs_expr)
            
solver.solve(model)

print(get_cycle(model, 'A'))

# Approach \#2
Initially solve the problem without the subtour elimination constraints. If the solution has subtours, add a cut that eliminates the subtour and resolve. Repeat until the solution is a single tour.

In [ ]:
model = pe.ConcreteModel()

model.V = pe.Set(initialize=V)
model.E = pe.Set(initialize=E)
model.c = pe.Param(model.E, initialize=c)
model.E_of_i = pe.Param(model.V, initialize=E_of_i)

model.x = pe.Var(model.E, domain=pe.Reals, bounds=(0, 1))

obj_expr = sum(model.c[e] * model.x[e] for e in model.E)
model.obj = pe.Objective(sense=pe.minimize, expr=obj_expr)

def con_flow_balance(model, i):
    return sum(model.x[e] for e in model.E_of_i[i]) == 2
model.con_flow_balance = pe.Constraint(model.V, rule=con_flow_balance)

model.con_no_subtours = pe.ConstraintList()
sol_edges = set()
while len(sol_edges) < len(model.V):
    solver = po.SolverFactory('gurobi')
    solver.solve(model)
    sol_edges = get_cycle(model, 'A')
    lhs = sum(model.x[e] for e in sol_edges)
    rhs = len(sol_edges) - 1
    model.con_no_subtours.add(lhs <= rhs)
    
print(sol_edges)

# Approach \#3
Make $\mathcal{V}$ an ordered set (so that we may index it) and use an index set of binary strings to represent the power set. Then we can create a rule that takes the model and the binary string.

In [ ]:
model = pe.ConcreteModel()

model.V = pe.Set(initialize=V, ordered=True)
model.E = pe.Set(initialize=E)
model.PowerSet = pe.Set(initialize=product([0, 1], repeat=len(model.V)))
model.c = pe.Param(model.E, initialize=c)
model.E_of_i = pe.Param(model.V, initialize=E_of_i)

model.x = pe.Var(model.E, domain=pe.Reals, bounds=(0, 1))

obj_expr = sum(model.c[e] * model.x[e] for e in model.E)
model.obj = pe.Objective(sense=pe.minimize, expr=obj_expr)

def con_flow_balance(model, i):
    return sum(model.x[e] for e in model.E_of_i[i]) == 2
model.con_flow_balance = pe.Constraint(model.V, rule=con_flow_balance)

def con_no_subtours(model, *binary_string):
    nodes = set(model.V[i] for i, bit in enumerate(binary_string, start=1) if bit)
    edges = set(e for i in nodes for e in model.E_of_i[i] if set(e).issubset(nodes))
    if 2 <= len(nodes) < len(model.V) and edges:
        constr = sum(model.x[e] for e in edges) <= len(nodes) - 1
    else:
        constr = pe.Constraint.Skip
    return constr
model.con_no_subtours = pe.Constraint(model.PowerSet, rule=con_no_subtours)

solver.solve(model)
cycle = get_cycle(model, 'A')
print(cycle)